# CUADERNO ENTRENAMIENTO MODELO

# Indice

1. [Importar Datos](#1.-Importar-Datos)
2. [Train Test](#2.-Train-Test)
3. [Scaler](#3.-Scaler)
4. [Pipe Line](#4.-Pipe-Line)
5. [GadientBoostingClassifier](#5.-GadientBoostingClassifier)
6. [Hiperparametrizacion GBC](#6.-Hiperparametrizacion-GBC)


In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, recall_score, accuracy_score
import pickle
import os

##  1. Importar Datos

[indice](#Indice)

In [2]:
url = "https://raw.githubusercontent.com/juanmendiz/ML-JUAN/master/data/Train.csv"
Train = pd.read_csv(url)

In [3]:
url = "https://raw.githubusercontent.com/juanmendiz/ML-JUAN/master/data/Test.csv"
Test = pd.read_csv(url)

## 2. Train Test

[indice](#Indice)

In [4]:
X_test = Test[['age', 'gender', 'height_cm', 'weight_kg', 'body fat_%', 'diastolic',
       'systolic', 'gripForce', 'sit and bend forward_cm', 'sit-ups counts',
       'broad jump_cm']]
y_test = Test['class']

In [5]:
X = Train[['age', 'gender', 'height_cm', 'weight_kg', 'body fat_%', 'diastolic',
       'systolic', 'gripForce', 'sit and bend forward_cm', 'sit-ups counts',
       'broad jump_cm']]

y = Train['class']

## 3. Scaler

[indice](#Indice)

from sklearn.metrics import make_scorer, recall_score

In [23]:
https://file+.vscode-resource.vscode-cdn.net/c%3A/Users/jamen/Documents/TheBridge/Alumno/JUAN%20ML/notebooks/#indicefrom sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.fit_transform(X_test)

In [19]:
print(X_scaled.shape)
print(X.shape)
print(X_test_scaled.shape)

(10714, 11)
(10714, 11)
(2679, 11)


## 4. Pipe Line
de ejercicio de Beast Cancer de 3.ML-1.supervisado -4.Pipeles-practica

[indice](#Indice)

In [46]:
pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ('classifier', RandomForestClassifier())
])

logistic_params = {
'selectkbest__k' : [1,2],
'classifier': [LogisticRegression(solver='liblinear')],
'classifier__penalty': ['l1','l2']
}

rf_params = {
'scaler' : [StandardScaler(), None],
'selectkbest__k' :[1,2],
'classifier': [RandomForestClassifier()],
'classifier__max_features': [2,3,4],
'classifier__max_depth': [3,5,7]
}

gb_params = {
'scaler' : [StandardScaler(), None],
'selectkbest__k' : [1,2],
'classifier': [GradientBoostingClassifier()],
'classifier__max_features': [2,3,4],
'classifier__max_depth': [3,5,7]
}

knn_params = {
'selectkbest__k' :[1,2],
'classifier': [KNeighborsClassifier()],
'classifier__n_neighbors': [1,2,3]
}

svm_params = {
'selectkbest__k' : [1,2],
'classifier': [SVC()],
'classifier__C': [20,40,60]
}

In [47]:
#  
search_space = [svm_params, logistic_params, rf_params, gb_params, knn_params]

clf = GridSearchCV(estimator = pipe, param_grid = search_space, cv=3, scoring="precision", n_jobs=-1)

In [48]:
clf.fit(X, y)

c:\Users\jamen\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
180 fits failed out of a total of 264.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
90 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\jamen\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\jamen\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\jamen\AppData\Local\Programs\Python\Python37\lib\site

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('selectkbest', SelectKBest()),
                                       ('classifier',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid=[{'classifier': [SVC(C=20)],
                          'classifier__C': [20, 40, 60],
                          'selectkbest__k': [1, 2]},
                         {'classifier': [LogisticRegression(solver='liblinear')],
                          'classifier__penalty': ['l1', 'l2'],
                          'selectkbest__k': [...
                          'classifier__max_features': [2, 3, 4],
                          'scaler': [StandardScaler(), None],
                          'selectkbest__k': [1, 2]},
                         {'classifier': [GradientBoostingClassifier()],
                          'classifier__max_depth': [3, 5, 7],
                       

In [49]:
print("Best Score", clf.best_score_)
print("\t")
print("Best Params", clf.best_params_)
print("\t")
print("Best Estimators", clf.best_estimator_)

Best Score nan
	
Best Params {'classifier': SVC(C=20), 'classifier__C': 20, 'selectkbest__k': 1}
	
Best Estimators Pipeline(steps=[('scaler', StandardScaler()), ('selectkbest', SelectKBest(k=1)),
                ('classifier', SVC(C=20))])


In [ ]:
# y_pred = clf.best_estimator_.predict(X_test)


## 5. GadientBoostingClassifier


[indice](#Indice)

In [7]:
seed = 0
scorer = make_scorer(recall_score, average='macro')
                     
model = GradientBoostingClassifier(n_estimators=100, random_state=seed)

gbc_cv = cross_val_score(model, X, y, cv=10, scoring= scorer,error_score='raise')

print('CV',gbc_cv)
print('CV media',gbc_cv.mean())
print('CV desv',gbc_cv.std())

CV [0.71604992 0.69073974 0.70845732 0.69533893 0.72316257 0.72879883
 0.72510508 0.73558237 0.69368322 0.72524809]
CV media 0.7142166077912704
CV desv 0.015346337058370938


In [8]:
model.fit(X, y)

GradientBoostingClassifier(random_state=0)

In [9]:
ruta_model = os.path.join("models","trained_model.pkl")

with open(ruta_model, 'wb') as archivo:
    pickle.dump(model, archivo)

FileNotFoundError: [Errno 2] No such file or directory: 'models\\trained_model.pkl'

In [34]:
y_pred = model.predict(X_test)

print("Accuracy", accuracy_score(y_test,y_pred))

Accuracy 0.7286300858529302


Accuracy sin Scaler: 0.7286
Accuracy MinMaxScaler: 0.6162
Accuracy StandarScaler: 0.7248

In [ ]:
import pickle

# Objeto a serializar
ruta_model = os.path.join("..","models","trained_model.pkl")
# Serializar objeto y guardarlo en un archivo
with open(ruta_model, 'wb') as archivo:
    pickle.dump(model, archivo)

# Deserializar objeto desde el archivo


# Imprimir el objeto deserializado
print(datos_deserializados)

In [41]:
# ruta_model = os.path.join("..","models","trained_model.pkl")


ruta_modelo = os.path.join('..','models','trained_model.pkl')

with open(ruta_modelo, "wb") as archivo:
    pickle.dump(model,archivo)

## 6. Hiperparametrizacion GBC

[indice](#Indice)

In [13]:
from sklearn.metrics import accuracy_score, recall_score

In [16]:
model = GradientBoostingClassifier()

parameters = {"n_estimators":[50,100,150],
              "max_depth":  [3,4,5],
              "learning_rate": [0.1,0.5,0.9],
              "min_samples_split": [4,8,12]}

gb_gs = GridSearchCV(model, parameters, cv=5, scoring="accuracy")

In [ ]:
gb_gs.fit(X, y)

In [15]:
print("Best Score", gb_gs.best_score_)
print("\t")
print("Best Params", gb_gs.best_params_)
print("\t")
print("Best Estimators", gb_gs.best_estimator_)

Best Score nan
	
Best Params {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_split': 4, 'n_estimators': 50}
	
Best Estimators GradientBoostingClassifier(min_samples_split=4, n_estimators=50)


In [16]:
# con Test
y_pred = gb_gs.best_estimator_.predict(X_test)

print("Accuracy", accuracy_score(y_test,y_pred))
print("Recall", recall_score(y_test,y_pred))

Accuracy 0.692795819335573


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].